In [1]:
from snorkel.labeling.model.label_model import LabelModel
import pandas as pd
import re
from hashlib import md5
from pathlib import Path
from transformers import AutoTokenizer
import datasets as ds
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
import datasets as ds
import json, os
import os, json, numpy as np, torch
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    DataCollatorForTokenClassification, TrainingArguments, Trainer,
    EarlyStoppingCallback
)
import datasets as ds
from seqeval.metrics import f1_score, precision_score, recall_score
import re, math
from collections import Counter, defaultdict


/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ROOT = Path.cwd().parent
df_lean_file = ROOT / "data" / "clean" / 'augmented_notes_30K.csv'
df_lean = pd.read_csv(df_lean_file).reset_index(drop=True)
all_entities_file = ROOT / "data" / "clean" / 'all_entities.csv'
all_entities = pd.read_csv(all_entities_file)

/var/folders/7m/wfwdc8s14pgch1s3hgyyg6080000gn/T/ipykernel_40097/4072654494.py:5: DtypeWarning: Columns (4,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_entities = pd.read_csv(all_entities_file)


In [3]:
df_lean

,conversation,full_note,idx,note,summary
0,"Doctor: Good morning, what brings you to the O...","A a sixteen year-old girl, presented to our Ou...",155216,"A a sixteen year-old girl, presented to our Ou...","{\n""visit motivation"": ""Discomfort in the neck..."
1,"Doctor: Hi, how are you feeling today?\nPatien...",This is the case of a 56-year-old man that was...,77465,This is the case of a 56-year-old man that was...,"{\n""visit motivation"": ""Complaints of a dull p..."
2,"Doctor: Hello, what brings you to the hospital...",A 36-year old female patient visited our hospi...,133948,A 36-year old female patient visited our hospi...,"{\n""visit motivation"": ""Pain and restricted ra..."
3,"Doctor: Good morning, Mr. [Patient's Name]. I'...",A 49-year-old male presented with a complaint ...,80176,A 49-year-old male presented with a complaint ...,"{\n""visit motivation"": ""Pain in the left proxi..."
4,"Doctor: Good morning, how are you feeling toda...",A 47-year-old male patient was referred to the...,72232,A 47-year-old male patient was referred to the...,"{\n""visit motivation"": ""Recurrent attacks of p..."
...,...,...,...,...,...
29995,"Doctor: Good morning, sir. Can you tell me wha...",A 28-year-old male was admitted to the emergen...,39279,A 28-year-old male was admitted to the emergen...,"{\n""visit motivation"": ""Stab wound under left ..."
29996,"Doctor: Good morning, sir. I am Dr. John. How ...","An 82-year-old man (64.5 kg, 175 cm) diagnosed...",137017,"An 82-year-old man (64.5 kg, 175 cm) diagnosed...","{\n""visit motivation"": ""Diagnosed with falcine..."
29997,"Doctor: Good morning, how are you feeling toda...",A 54 year-old man with no past medical history...,98004,A 54 year-old man with no past medical history...,"{\n""visit motivation"": ""Cardiac sounding chest..."
29998,"Doctor: Good morning, how can I help you today...",A 49-year-old woman visited the clinic due to ...,133320,A 49-year-old woman visited the clinic due to ...,"{\n""visit motivation"": ""Mass in her right thig..."


In [4]:
df_lean[df_lean['idx'] == 100017]

,conversation,full_note,idx,note,summary
8900,"Doctor: Hi, I'm Dr. X. Can you tell me what ha...",A 42-year-old woman with a past medical histor...,100017,A 42-year-old woman with a past medical histor...,"{\n""visit motivation"": ""Mechanical fall"",\n""ad..."


In [5]:
all_entities = all_entities.rename(columns={"row_idx": "idx"})

all_entities = all_entities[["idx","text","label","table","original_text"]].copy()

In [6]:
all_entities

,idx,text,label,table,original_text
0,80176,posttraumatic arthritis,DISEASE,physiological,History of left elbow arthrodesis performed fo...
1,31864,pain,DISEASE,physiological,"Inability to walk since babyhood, did not walk..."
2,31864,fracture,DISEASE,physiological,"Inability to walk since babyhood, did not walk..."
3,149866,Coxa vara deformity,DISEASE,physiological,"Coxa vara deformity of bilateral hips, bilater..."
4,149866,fracture,DISEASE,physiological,"Coxa vara deformity of bilateral hips, bilater..."
...,...,...,...,...,...
2113052,97973,31 y/o men,AgeSex,info,NaN
2113053,97973,aged 31 male,AgeSex,info,NaN
2113054,97973,aged 31 man,AgeSex,info,NaN
2113055,97973,aged 31 male patient,AgeSex,info,NaN


In [7]:
all_entities['idx'].head()

0     80176
1     31864
2     31864
3    149866
4    149866
Name: idx, dtype: int64

In [8]:
all_entities['label'].unique()

array(['DISEASE', 'CHEMICAL', 'ANATOMY', 'ANATOMY_WITH_LATERALITY',
       'PATHOLOGICAL_FORMATION', 'MULTI_TISSUE_STRUCTURE', 'TISSUE',
       'ORGANISM_SUBDIVISION', 'CELL', 'ORGAN', 'CANCER',
       'ORGANISM_SUBSTANCE', 'ORGANISM', 'IMMATERIAL_ANATOMICAL_ENTITY',
       'GENE_OR_GENE_PRODUCT', 'SIMPLE_CHEMICAL', 'CELLULAR_COMPONENT',
       'AMINO_ACID', 'ANATOMICAL_SYSTEM',
       'DEVELOPING_ANATOMICAL_STRUCTURE', 'PROCEDURE', 'LATERALITY',
       'SYMPTOM', 'SYMPTOM_TYPE', 'SEVERITY', 'TEMPORAL_PATTERN', 'TEST',
       'TEST_TYPE', 'FINDING', 'CONDITION', 'MEASUREMENT', 'TREATMENT',
       'TREATMENT_TYPE', 'MEDICATION', 'DOSAGE', 'FREQUENCY', 'ROUTE',
       'TREATMENT_RESPONSE', 'CONDITION_TYPE', 'TREATMENT_REASON', 'Age',
       'Sex', 'AgeSex'], dtype=object)

In [9]:
# pick the text the model should read
TEXT_COL = "note"         
assert "idx" in df_lean.columns, "df must contain idx"
assert TEXT_COL in df_lean.columns, f"'{TEXT_COL}' not found in df"

df_lean = df_lean[["idx", TEXT_COL]].dropna(subset=[TEXT_COL]).copy()
df_lean[TEXT_COL] = df_lean[TEXT_COL].astype(str)

# sanity check for all_entities
need = {"idx","text","label"}
missing = need.difference(all_entities.columns)
assert not missing, f"all_entities is missing columns: {missing}"

#### Normalize labels & phrases

In [10]:
# Map upstream labels to the schema
LABEL_MAP = {
    # Problems / conditions
    "DISEASE": "Problem",
    "SYMPTOM": "Problem",
    "CONDITION": "Problem",
    "CONDITION_TYPE": "Problem",
    "CANCER": "Problem",
    "FINDING": "Problem",          

    # Anatomy (broad)
    "ANATOMY": "Anatomy",
    "ANATOMY_WITH_LATERALITY": "Anatomy",  # keep as Anatomy; Laterality is its own span
    "TISSUE": "Anatomy",
    "MULTI_TISSUE_STRUCTURE": "Anatomy",
    "ORGAN": "Anatomy",
    "ANATOMICAL_SYSTEM": "Anatomy",
    "ORGANISM_SUBDIVISION": "Anatomy",
    "DEVELOPING_ANATOMICAL_STRUCTURE": "Anatomy",
    "IMMATERIAL_ANATOMICAL_ENTITY": "Anatomy",
    "CELL": "Anatomy",
    "CELLULAR_COMPONENT": "Anatomy",

    # Medications / substances
    "CHEMICAL": "Medication",
    "SIMPLE_CHEMICAL": "Medication",
    "MEDICATION": "Medication",

    # Other bio entities (keep separate in case you want to use later; otherwise map to 'Anatomy' or drop)
    "GENE_OR_GENE_PRODUCT": "Substance",
    "AMINO_ACID": "Substance",
    "ORGANISM": "Substance",
    "ORGANISM_SUBSTANCE": "Substance",

    # Procedures & tests
    "PROCEDURE": "Procedure",
    "TEST": "TestName",
    "TEST_TYPE": "TestType",

    # Attributes / modifiers
    "LATERALITY": "Laterality",
    "SEVERITY": "Severity",
    "TEMPORAL_PATTERN": "TemporalPattern",
    "MEASUREMENT": "Measurement",

    # Treatment-related (keep distinct; useful for relations)
    "TREATMENT": "TreatmentName",          # generic treatment/therapy names
    "TREATMENT_TYPE": "TreatmentType",
    "TREATMENT_RESPONSE": "TreatmentResponse",
    "TREATMENT_REASON": "TreatmentReason",

    # Dosing
    "DOSAGE": "Dosage",
    "FREQUENCY": "Frequency",
    "ROUTE": "Route",

    # Symptom subtype -> usually just a Problem
    "SYMPTOM_TYPE": "Problem",

    # Information-related (keep distinct; useful for relations)
    "AGE": "Age",
    "SEX": "Sex",
    "AGESEX": "AgeSex",
}
ALLOWED_LABELS = {
    "Problem","Anatomy","Medication","Procedure",
    "TestName","TestType","Laterality","Severity","TemporalPattern",
    "Measurement","Dosage","Frequency","Route",
    "TreatmentName","TreatmentType","TreatmentResponse","TreatmentReason",
    "Age","Sex","AgeSex"}

def normalize_label(lbl: str) -> str:
    if not isinstance(lbl, str): 
        return ""
    base = lbl.strip().upper()
    # map Age/Sex variants
    if base == "AGE": return "Age"
    if base == "SEX": return "Sex"
    if base in ("AGESEX","AGE_SEX"): return "AgeSex"
    mapped = LABEL_MAP.get(base, base)
    return mapped

def normalize_phrase(s: str) -> str:
    return re.sub(r"\s+", " ", str(s).strip())

# keep only phrase-like rows, keep idx intact
ae = all_entities.dropna(subset=["text","label"]).copy()
ae["text"]  = ae["text"].astype(str).map(normalize_phrase)
ae["label"] = ae["label"].astype(str).map(normalize_label)

# filter to allowed schema and remove super short tokens
ae = ae[(ae["text"].str.len() >= 2) & (ae["label"].isin(ALLOWED_LABELS))]

# de-duplicate
ae = ae[["idx","text","label"]].drop_duplicates()

print("Candidate phrases:", ae.shape)
print("\nSchema label distribution:\n", ae["label"].value_counts().head(20))

# Debug: see which upstream labels were dropped or mapped to unknown
upstream_leftovers = (
    all_entities.assign(__norm=all_entities["label"].astype(str).map(lambda x: LABEL_MAP.get(x.strip().upper(), x.strip().upper())))
    .loc[~all_entities["label"].astype(str).map(lambda x: LABEL_MAP.get(x.strip().upper(), x.strip().upper()) in ALLOWED_LABELS), "label"]
    .astype(str).str.upper().value_counts()
)
print("\nUpstream labels not kept (after mapping):\n", upstream_leftovers.head(20))


Candidate phrases: (1805939, 3)

Schema label distribution:
 label
AgeSex               707568
Problem              372982
Age                  203321
Anatomy              147999
TestName              55700
Medication            46450
TreatmentName         44086
TestType              42025
Laterality            41044
TemporalPattern       25025
TreatmentType         23652
Procedure             19639
Route                 18590
TreatmentReason       16764
Dosage                11733
Measurement            9676
TreatmentResponse      7463
Frequency              6705
Severity               5509
Sex                       8
Name: count, dtype: int64

Upstream labels not kept (after mapping):
 label
PATHOLOGICAL_FORMATION    14321
GENE_OR_GENE_PRODUCT       3097
ORGANISM                   2378
ORGANISM_SUBSTANCE         1306
AMINO_ACID                  111
Name: count, dtype: int64


#### Match candidate phrases back into the note text → silver spans

In [11]:
def find_all(text: str, phrase: str):
    """All case-insensitive exact matches (start,end)."""
    return [(m.start(), m.end()) for m in re.finditer(re.escape(phrase), text, flags=re.IGNORECASE)]

def dedupe_overlaps(spans):
    """
    Keep one span per overlapping region per label (prefer longer).
    spans: list of dicts [{start,end,label,text}, ...]
    """
    spans = sorted(spans, key=lambda x: (x["label"], x["start"], -(x["end"]-x["start"])))
    kept = []
    for sp in spans:
        conflict = False
        for kp in kept:
            if sp["label"] == kp["label"] and not (sp["end"] <= kp["start"] or sp["start"] >= kp["end"]):
                conflict = True
                # prefer longer
                if (sp["end"]-sp["start"]) > (kp["end"]-kp["start"]):
                    kp.update(sp)
                break
        if not conflict:
            kept.append(sp)
    return kept

# pre-group for speed
ents_by_idx = {k: g[["text","label"]].to_dict("records") for k,g in ae.groupby("idx")}

silver_rows = []
matched_notes = 0

for r in df_lean.itertuples(index=False):
    note_id = r.idx
    text    = getattr(r, TEXT_COL)
    spans = []
    for ent in ents_by_idx.get(note_id, []):
        for (s,e) in find_all(text, ent["text"]):
            spans.append({"start": s, "end": e, "label": ent["label"], "text": text[s:e]})
    spans = dedupe_overlaps(spans)
    matched_notes += int(len(spans) > 0)
    silver_rows.append({"idx": note_id, "text": text, "silver_spans": spans})

silver_df = pd.DataFrame(silver_rows)
silver_df["n_spans"] = silver_df["silver_spans"].str.len()
print(f"Notes with ≥1 span: {matched_notes}/{len(silver_df)} = {matched_notes/len(silver_df):.1%}")

Notes with ≥1 span: 29776/30000 = 99.3%


#### Quick audit / save silver spans

In [12]:
# label distribution
lab_counts = Counter(sp["label"] for L in silver_df["silver_spans"] for sp in L)
print("Top labels in matched spans:", lab_counts.most_common(10))

# sample for eyeballing
sample = silver_df[silver_df["n_spans"]>0].sample(5, random_state=13)
for row in sample.itertuples(index=False):
    print("\n--- idx:", row.idx, " ---")
    print(row.text[:250], "…")
    print("spans:", row.silver_spans[:4])

# save for reuse
silver_df.to_parquet("silver_spans.parquet", index=False)

Top labels in matched spans: [('Problem', 322501), ('Anatomy', 236202), ('TestType', 105973), ('Laterality', 103713), ('Medication', 56871), ('TestName', 53364), ('Age', 29947), ('Procedure', 24026), ('AgeSex', 17599), ('TemporalPattern', 15126)]

--- idx: 57947  ---
A 64-year-old caucasian woman with a past medical history of scleroderma with pulmonary fibrosis, renal transplant 18 years ago, chronic pericardial effusion, and hypertension presented to the hospital with two days of right-sided neck pain and stiff …
spans: [{'start': 2, 'end': 13, 'label': 'Age', 'text': '64-year-old'}, {'start': 78, 'end': 87, 'label': 'Anatomy', 'text': 'pulmonary'}, {'start': 98, 'end': 103, 'label': 'Anatomy', 'text': 'renal'}, {'start': 219, 'end': 235, 'label': 'Anatomy', 'text': 'right-sided neck'}]

--- idx: 46382  ---
We describe a case of a 78-year-old male who 24 hours after an anterior rectum surgical resection with a terminoterminal anastomosis for a rectal neoplasm,showed sudden pain among

In [13]:
%pip install rapidfuzz

import re
from collections import defaultdict
from rapidfuzz import fuzz

# explode to candidate rows
rows = []
for r in silver_df.itertuples(index=False):
    for sp in r.silver_spans:
        rows.append({"idx": r.idx, "note_text": r.text, **sp})
cand = pd.DataFrame(rows)

# simple priors: frequency + local fuzzy + case exact
freq = (cand.groupby(["label","text"]).size()/len(silver_df)).to_dict()

def conf_score(row):
    s = 0.0
    s += 0.45 * freq.get((row["label"], row["text"]), 0)
    s += 0.25 * (1.0 if row["note_text"][row["start"]:row["end"]] == row["text"] else 0.0)
    local = row["note_text"][max(0,row["start"]-40):min(len(row["note_text"]),row["end"]+40)]
    s += 0.30 * (fuzz.partial_ratio(row["text"].lower(), local.lower())/100.0)
    return min(s,1.0)

cand["conf"] = cand.apply(conf_score, axis=1)
cand = cand[cand["conf"] >= 0.35]

# rebuild silver_df_filtered
def rebuild(group):
    spans = group[["start","end","label","text"]].to_dict("records")
    return pd.Series({"text": group["note_text"].iloc[0], "silver_spans": spans})

silver_df = cand.groupby("idx").apply(rebuild).reset_index()
silver_df["n_spans"] = silver_df["silver_spans"].str.len()
print("Notes with ≥1 span:", (silver_df["n_spans"]>0).mean())
silver_df.to_parquet("silver_spans_filtered.parquet", index=False)

Note: you may need to restart the kernel to use updated packages.


/var/folders/7m/wfwdc8s14pgch1s3hgyyg6080000gn/T/ipykernel_40097/3921894254.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  silver_df = cand.groupby("idx").apply(rebuild).reset_index()


Notes with ≥1 span: 1.0


#### Convert silver spans → BIO sequences for Hugging Face NER

In [14]:
# 4.1 label space
entity_labels = sorted({sp["label"] for L in silver_df["silver_spans"] for sp in L})
id2label = ["O"] + sum(([f"B-{l}", f"I-{l}"] for l in entity_labels), [])
label2id = {l:i for i,l in enumerate(id2label)}
print("Entity set:", entity_labels)

# 4.2 tokenizer
MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
MAX_LEN = 256
tok = AutoTokenizer.from_pretrained(MODEL_NAME)

def to_bio(example):
    text, spans = example["text"], example["silver_spans"]
    enc = tok(text, truncation=True, max_length=MAX_LEN, return_offsets_mapping=True)
    tags = ["O"] * len(enc["offset_mapping"])

    for sp in spans:
        s, e, lab = sp["start"], sp["end"], sp["label"]
        began = False
        for i, (a, b) in enumerate(enc["offset_mapping"]):
            if a == b:         # special tokens like [CLS]/[SEP]
                continue
            if a >= s and b <= e:
                tags[i] = f"I-{lab}" if began else f"B-{lab}"
                began = True

    enc["labels"] = [label2id[t] for t in tags]
    enc["idx"] = example["idx"]         # <— keep idx in the HF example
    enc.pop("offset_mapping", None)
    return enc

# 4.3 build HF dataset (keeping idx)
hf = ds.Dataset.from_pandas(silver_df[["idx","text","silver_spans"]], preserve_index=False)
hf = hf.map(to_bio, remove_columns=["text","silver_spans"], desc="Tokenize + BIO")

# 4.4 split & save (idx stays inside the dataset)
splits = hf.train_test_split(test_size=0.15, seed=42)
splits.save_to_disk("ds_ner_silver_with_idx")

# persist label maps for training/eval later
with open("ner_label_space.json","w") as f:
    json.dump({"id2label": id2label, "label2id": label2id}, f, indent=2)

print(splits)

Entity set: ['Age', 'AgeSex', 'Anatomy', 'Dosage', 'Frequency', 'Laterality', 'Measurement', 'Medication', 'Problem', 'Procedure', 'Route', 'Severity', 'Sex', 'TemporalPattern', 'TestName', 'TestType', 'TreatmentName', 'TreatmentReason', 'TreatmentResponse', 'TreatmentType']


Saving the dataset (1/1 shards): 100%|██████████| 4467/4467 [00:00<00:00, 113064.73 examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 25309
    })
    test: Dataset({
        features: ['idx', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4467
    })
})


#### Train a NER model (HF Trainer)

In [15]:
import os, json, numpy as np, torch, datasets as ds
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    DataCollatorForTokenClassification, TrainingArguments, Trainer,
    EarlyStoppingCallback, __version__ as HF_VER
)
from seqeval.metrics import f1_score, precision_score, recall_score

print("🤖 transformers version:", HF_VER)

# --- Data ---
splits = ds.load_from_disk("ds_ner_silver_with_idx")
if "labels" not in splits["train"].features:
    if "ner_tags" in splits["train"].features:
        splits = splits.rename_column("ner_tags", "labels")
    else:
        raise ValueError(f"Need a 'labels' (or 'ner_tags') column. Got: {list(splits['train'].features)}")

with open("ner_label_space.json") as f:
    maps = json.load(f)

id2label_list = maps["id2label"]
# normalize types for robustness
label2id = {str(k): int(v) for k, v in maps["label2id"].items()}
id2label = {i: l for i, l in enumerate(id2label_list)}

# --- Model / Tokenizer ---
MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# --- TrainingArguments (version-safe) ---
use_cuda = torch.cuda.is_available()
TA_fields = set(TrainingArguments.__dataclass_fields__.keys())
ta = dict(
    output_dir="ckpt/ner",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
)

# warmup_ratio if supported
if "warmup_ratio" in TA_fields:
    ta["warmup_ratio"] = 0.1
# fp16 only if supported
if "fp16" in TA_fields:
    ta["fp16"] = use_cuda
# report_to if supported
if "report_to" in TA_fields:
    ta["report_to"] = "none"
# save_total_limit if supported
if "save_total_limit" in TA_fields:
    ta["save_total_limit"] = 2

# Evaluation/saving strategy compatibility
if "evaluation_strategy" in TA_fields and "save_strategy" in TA_fields:
    ta["evaluation_strategy"] = "epoch"
    ta["save_strategy"] = "epoch"
elif "evaluate_during_training" in TA_fields:
    # very old versions use this boolean instead
    ta["evaluate_during_training"] = True
    # fall back to step-based saving if epoch strategy isn't available
    if "save_steps" in TA_fields:
        ta["save_steps"] = 500

# Best-model logic compatibility
if "load_best_model_at_end" in TA_fields:
    ta["load_best_model_at_end"] = True
    # metric_for_best_model name must match the logged key (eval_f1)
    if "metric_for_best_model" in TA_fields:
        ta["metric_for_best_model"] = "eval_f1"
    if "greater_is_better" in TA_fields:
        ta["greater_is_better"] = True

# --- ensure eval/save strategies MATCH across HF versions ---
TA_FIELDS = set(TrainingArguments.__dataclass_fields__.keys())

# pick the correct key name for evaluation strategy
eval_key = "eval_strategy" if "eval_strategy" in TA_FIELDS else (
    "evaluation_strategy" if "evaluation_strategy" in TA_FIELDS else None
)

if eval_key:
    ta[eval_key] = "epoch"        # evaluate each epoch
if "save_strategy" in TA_FIELDS:
    ta["save_strategy"] = "epoch" # save each epoch

# remove step-based settings so they don't force 'steps' behavior
for k in ("save_steps", "eval_steps"):
    if k in ta:
        ta.pop(k)
args = TrainingArguments(**ta)

# --- Metrics ---
def align_and_decode(preds, labels):
    pred_labels, true_labels = [], []
    for p, t in zip(preds, labels):
        p = np.array(p); t = np.array(t)
        mask = t != -100
        p = p[mask]; t = t[mask]
        pred_labels.append([id2label[i] for i in p])
        true_labels.append([id2label[i] for i in t])
    return pred_labels, true_labels

def compute_metrics(p):
    logits, labels = p.predictions, p.label_ids
    preds = np.argmax(logits, axis=-1)
    pred_tags, true_tags = align_and_decode(preds, labels)
    return {
        "precision": precision_score(true_tags, pred_tags),
        "recall":    recall_score(true_tags, pred_tags),
        "f1":        f1_score(true_tags, pred_tags),
    }

# --- Trainer ---
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=splits["train"],
    eval_dataset=splits.get("validation", splits.get("test")),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

train_output = trainer.train()
eval_output  = trainer.evaluate()
print(eval_output)

trainer.save_model("model_ner_best")
tokenizer.save_pretrained("model_ner_best")


🤖 transformers version: 4.55.2


Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/7m/wfwdc8s14pgch1s3hgyyg6080000gn/T/ipykernel_40097/42040209.py:129: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.252400,0.247244,0.559288,0.716138,0.628069
2,0.220300,0.232989,0.582861,0.742173,0.652939
3,0.205900,0.223240,0.606255,0.733174,0.663702


/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.22324031591415405, 'eval_precision': 0.6062553130416543, 'eval_recall': 0.7331740076636537, 'eval_f1': 0.6637015192351853, 'eval_runtime': 284.0717, 'eval_samples_per_second': 15.725, 'eval_steps_per_second': 0.986, 'epoch': 3.0}


('model_ner_best/tokenizer_config.json',
 'model_ner_best/special_tokens_map.json',
 'model_ner_best/vocab.txt',
 'model_ner_best/added_tokens.json',
 'model_ner_best/tokenizer.json')

#### Detailed Eval - Seqeval

In [16]:
from sklearn.metrics import classification_report


pred = trainer.predict(splits["test"])
pred_ids = pred.predictions.argmax(-1)

def align(ids, gold):
    p_tags, t_tags = [], []
    for p, t in zip(ids, gold):
        mask = (t != -100)
        p = p[mask]; t = t[mask]
        p_tags.append([id2label[i] for i in p])
        t_tags.append([id2label[i] for i in t])
    return p_tags, t_tags

p_tags, t_tags = align(pred_ids, pred.label_ids)
# Flatten the lists of tags for sklearn compatibility
flat_true = [tag for seq in t_tags for tag in seq]
flat_pred = [tag for seq in p_tags for tag in seq]
print(classification_report(flat_true, flat_pred, digits=3))

/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                     precision    recall  f1-score   support

              B-Age      0.919     0.947     0.933      1697
           B-AgeSex      0.993     1.000     0.996      2680
          B-Anatomy      0.589     0.590     0.590     12177
        B-Frequency      0.000     0.000     0.000         8
       B-Laterality      0.834     0.961     0.893      8580
      B-Measurement      0.552     0.682     0.610       726
       B-Medication      0.772     0.848     0.808      3456
          B-Problem      0.689     0.760     0.723     29669
        B-Procedure      0.819     0.960     0.884      1607
            B-Route      0.000     0.000     0.000        42
         B-Severity      0.643     0.839     0.728       818
  B-TemporalPattern      0.753     0.926     0.831      1663
         B-TestName      0.535     0.541     0.538      2550
         B-TestType      0.631     0.833     0.718      4715
    B-TreatmentName      0.000     0.000     0.000         1
B-TreatmentResponse    

/opt/anaconda3/envs/nlp_clinical/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Inference Helper

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch, pandas as pd, json, re

# ---- model & pipeline ----
device = 0 if torch.cuda.is_available() else -1
tok   = AutoTokenizer.from_pretrained("model_ner_best")
model = AutoModelForTokenClassification.from_pretrained("model_ner_best")
ner_pipe = pipeline("token-classification", model=model, tokenizer=tok,
                    aggregation_strategy="simple", device=device)

# ---- helpers for windowed mode ----
def merge_overlaps(ents):
    ents = sorted(ents, key=lambda e: (e["start"], e["end"]))
    merged = []
    for e in ents:
        if merged and e["start"] <= merged[-1]["end"] and e["entity_group"] == merged[-1]["entity_group"]:
            merged[-1]["end"]   = max(merged[-1]["end"], e["end"])
            merged[-1]["score"] = max(merged[-1]["score"], e["score"])
            # merged[-1]["word"] can be recomputed from text if you want exact text
        else:
            merged.append(e)
    return merged

def ner_with_windows(text, max_length=512, stride=128):
    enc = tok(
        text, return_offsets_mapping=True, return_overflowing_tokens=True,
        truncation=True, max_length=max_length, stride=stride
    )
    all_ents = []
    for offsets in enc["offset_mapping"]:
        valid = [(a, b) for (a, b) in offsets if b > a]
        if not valid:
            continue
        start_char = valid[0][0]
        end_char   = valid[-1][1]
        chunk      = text[start_char:end_char]
        ents = ner_pipe(chunk)
        for e in ents:
            e["start"] += start_char
            e["end"]   += start_char
        all_ents.extend(ents)
    return merge_overlaps(all_ents)

# ---- unified runner ----
def run_ner(df_notes, text_col, strategy="auto", max_length=512, stride=128):
    """
    strategy:
      - 'truncate' : fast, may drop content > max_length
      - 'window'   : slower, covers full text with overlap
      - 'auto'     : use windowed only when needed (len(tokens) > max_length)
    """
    rows = []
    for r in df_notes.itertuples(index=False):
        text = getattr(r, text_col) or ""
        if strategy == "truncate":
            ents = ner_pipe(text)
        elif strategy == "window":
            ents = ner_with_windows(text, max_length=max_length, stride=stride)
        else:  # auto
            # estimate token count; if close to limit, switch to windowed
            approx_len = len(tok(text, add_special_tokens=False)["input_ids"])
            if approx_len > max_length - 8:
                ents = ner_with_windows(text, max_length=max_length, stride=stride)
            else:
                ents = ner_pipe(text)
        rows.append({"idx": r.idx, "text": text, "pred_entities": ents})
    return pd.DataFrame(rows)

# Fast smoke test:
pred_df = run_ner(df_lean[["idx","note"]].head(10), "note", strategy="truncate")

# Full dataset, robust:
# pred_df = run_ner(df_lean[["idx","note"]], "note", strategy="window")

# Auto mode (fast on short notes, robust on long ones):
pred_df = run_ner(df_lean[["idx","note"]], "note", strategy="auto")

pred_df.to_parquet("ner_predictions.parquet", index=False)

# Convert all numpy types to native Python types for JSON serialization
import numpy as np
def convert_types(obj):
    if isinstance(obj, (np.generic, np.floating, np.integer)):
        return obj.item()
    return obj
# ---- JSON-safe conversion helpers ----
import torch
def to_jsonable(x):
    # numpy scalars -> Python scalars
    if isinstance(x, np.generic):
        return x.item()
    # numpy arrays -> lists
    if isinstance(x, np.ndarray):
        return x.tolist()
    # torch tensors -> lists
    if torch.is_tensor(x):
        return x.detach().cpu().tolist()
    # pandas timestamps/timedeltas -> ISO strings
    if isinstance(x, pd.Timestamp):
        return x.isoformat()
    # recurse into containers
    if isinstance(x, dict):
        return {k: to_jsonable(v) for k, v in x.items()}
    if isinstance(x, (list, tuple, set)):
        return [to_jsonable(v) for v in x]
    return x

records = pred_df.head(20).applymap(convert_types).to_dict(orient="records")
with open("ner_predictions_sample.json", "w") as f:
    json.dump(
        pred_df.head(20).to_dict(orient="records"),
        f,
        indent=2,
        allow_nan=False,
        default=to_jsonable  # will be invoked for np.float32, tensors, etc.
    )


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/var/folders/7m/wfwdc8s14pgch1s3hgyyg6080000gn/T/ipykernel_40097/867186477.py:108: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  records = pred_df.head(20).applymap(convert_types).to_dict(orient="records")


##### Post-process to structured JSON (merges & med bundles)

In [18]:
import re
def sent_split(text): return re.split(r'(?<=[\.\!\?])\s+|\n+', text)

def to_structured(note_text, ents):
    ents_sorted = sorted(ents, key=lambda x: (x["start"], x["end"]))
    merged=[]; i=0
    while i<len(ents_sorted):
        e=ents_sorted[i]
        if e["entity_group"]=="Laterality" and i+1<len(ents_sorted) and ents_sorted[i+1]["entity_group"]=="Anatomy":
            nxt=ents_sorted[i+1]
            merged.append({"entity_group":"Anatomy","start":e["start"],"end":nxt["end"],
                           "word":note_text[e["start"]:nxt["end"]].strip(),
                           "score":(e["score"]+nxt["score"])/2})
            i+=2
        else:
            merged.append(e); i+=1

    sents = sent_split(note_text); bounds=[]; pos=0
    for s in sents: bounds.append((pos,pos+len(s))); pos+=len(s)+1
    def sid(s): 
        for k,(a,b) in enumerate(bounds):
            if a<=s<b: return k
        return -1
    for e in merged: e["_sid"]=sid(e["start"])

    meds=[]
    for m in [x for x in merged if x["entity_group"]=="Medication"]:
        s=m["_sid"]
        meds.append({
            "name": m["word"], "span":[m["start"],m["end"]],
            "dosage":[x["word"] for x in merged if x["_sid"]==s and x["entity_group"]=="Dosage"],
            "frequency":[x["word"] for x in merged if x["_sid"]==s and x["entity_group"]=="Frequency"],
            "route":[x["word"] for x in merged if x["_sid"]==s and x["entity_group"]=="Route"]
        })
    problems=[x for x in merged if x["entity_group"]=="Problem"]
    anatomy=[x for x in merged if x["entity_group"]=="Anatomy"]
    temporal=[x for x in merged if x["entity_group"]=="TemporalPattern"]

    return {"problems":[{"text":p["word"],"span":[p["start"],p["end"]]} for p in problems],
            "anatomy":[{"text":a["word"],"span":[a["start"],a["end"]]} for a in anatomy],
            "temporal":[{"text":t["word"],"span":[t["start"],t["end"]]} for t in temporal],
            "medications": meds}

##### Relation candidates (by sentence, typed)

In [19]:
from itertools import product
import pandas as pd

def relation_candidates(note_text, merged_ents):
    # expects entities like in previous cell (merged)
    # assign sid
    sents = sent_split(note_text); bounds=[]; pos=0
    for s in sents: bounds.append((pos,pos+len(s))); pos+=len(s)+1
    def sid(s):
        for k,(a,b) in enumerate(bounds):
            if a<=s<b: return k
        return -1
    for e in merged_ents: e["_sid"]=sid(e["start"])

    types_ok = {
        ("Problem","Anatomy"): "has_location",
        ("Problem","TemporalPattern"): "has_duration",
        ("Medication","Dosage"): "has_dosage",
        ("Medication","Frequency"): "has_frequency",
        ("Medication","Route"): "has_route",
        ("Medication","Problem"): "treated_for",
    }
    rels=[]
    for i,h in enumerate(merged_ents):
        for j,t in enumerate(merged_ents):
            if j<=i: continue
            if h["_sid"]==-1 or t["_sid"]==-1 or h["_sid"]!=t["_sid"]: continue
            k=(h["entity_group"], t["entity_group"])
            if k in types_ok:
                rels.append({"type":types_ok[k], "head":i, "tail":j})
            k=(t["entity_group"], h["entity_group"])
            if k in types_ok:
                rels.append({"type":types_ok[k], "head":j, "tail":i})
    return rels

##### Run model end-to-end on a sample

In [20]:
# Use the same text column you trained on (e.g., 'note' or 'full_note')
sample = df_lean[["idx","note"]].head(10).copy()
pred = run_ner(sample, "note")

structured_rows=[]
for r in pred.itertuples(index=False):
    st = to_structured(r.text, r.pred_entities)
    # prepare flat entity list for relation candidates
    flat = []
    for pe in r.pred_entities:
        flat.append({"entity_group":pe["entity_group"],"start":pe["start"],"end":pe["end"],"word":pe["word"]})
    rels = relation_candidates(r.text, flat)
    structured_rows.append({"idx": r.idx, "entities": r.pred_entities, "structured": st, "relations": rels})

pd.DataFrame(structured_rows).to_json("inference_sample.json", orient="records", lines=False, indent=2)